In [380]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

from flask_cors import CORS

import pandas as pd

import os
import requests
import io

In [381]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['Actual Bookings']=df['Actual Bookings'].astype('float')
df['Ramp-up frontier']=df['Ramp-up frontier'].astype('float')
df['Ideal curve (80% LF)']=df['Ideal curve (80% LF)'].astype('float')
df['Ideal curve (100% LF)']=df['Ideal curve (100% LF)'].astype('float')
df['Phase-down frontier']=df['Phase-down frontier'].astype('float')

dep_dates = sorted(df['DepDate'].unique())
dtime = sorted(df['dtime'].unique())

# Function for conditional coloring

In [382]:
COLORS = [
    
        {
        'background': '#E8E8E8',
        'text': 'white'
    },
    
    {
        'background': '#FF0000',
        'text': 'black'
    },
    {
        'background': '#5DFC0A',
        'text': 'black'
    },
]

def cell_style(value1, value2, value3):

    # style = {}
    relative_value1 = value1 - value2
    relative_value2 = value1 - value3

    if relative_value1 < 0 or relative_value2 > 0:
            style = {
                'backgroundColor': COLORS[1]['background'],
                'color': COLORS[1]['text']
            }
    else:
            style = {
                'backgroundColor': COLORS[2]['background'],
                'color': COLORS[2]['text']
            }
    return style

# Function for table creation

In [383]:
def generate_table(dataframe, max_rows=50):
    rows = []
    for i in range(min(len(dataframe), max_rows)):
        row = []
        for col in dataframe.columns:
            if col == 'NumPax':
                value1 = dataframe.iloc[i][col]
                value2 = dataframe.iloc[i]['Ramp-up frontier']
                value3 = dataframe.iloc[i]['Phase-down frontier']
                style = cell_style(value1, value2, value3)
                row.append(html.Td(value1, style=style))
            else:
                value = dataframe.iloc[i][col]
                style = style = {
                'backgroundColor': COLORS[0]['background'],
                'color': COLORS[0]['text']
            }
                row.append(html.Td(value, style = style))
        rows.append(html.Tr(row))

    return html.Table(
            # Header
            [html.Tr([html.Th(col) for col in dataframe.columns])] +

            # Body
            rows)

# Import external stylesheet

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Layout

In [ ]:
line_columns = ['Actual Bookings', 'Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 'Phase-down frontier']
linecolor_dict = dict(zip(line_columns,['black', 'red', 'blue', 'green', 'red']))
linedash_dict = dict(zip(line_columns,['solid', 'dot', 'solid', 'solid', 'dot']))
name_dict = dict(zip(line_columns,['Actual Bookings', 'Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 'Phase-down frontier']))

In [ ]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    # Header
    html.H1(children='Booking curve monitoring system',style={
            'textAlign': 'center',
            'color': 'black'
        }),
    # Dropdown for departure dates
    html.Div([html.Div([dcc.Dropdown(id = 'dropdown_depdates',
    options=[{
    'label': i,
    'value': i
    } for i in dep_dates],
    value='All Departure Dates')],
    style={'width': '24%', 'display': 'inline-block'}),
    # Dropdown for departure times
    html.Div([dcc.Dropdown(id = 'dropdown_dtime',
    options=[{
    'label': i,
    'value': i
    } for i in dtime],
    value='All Departure Times')],
    style={'width': '24%', 'display': 'inline-block'})]),
    html.Div([html.Div(id='table-container',style={'width': '48%', 'display': 'inline-block'}),
    html.Div([dcc.Graph(id='graph-with-markdown')],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})]),
])

In [ ]:
@app.callback(
    # Output('table-container', 'children'),
    Output('graph-with-markdown', 'figure'),
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_graph(value_depdates, value_dtime):

    dff = df.loc[(df['DepDate'] == value_depdates)&(df['dtime'] == value_dtime),:] # update with your own logic    
    
    traces = []
    for i in line_columns:
        traces.append(go.Scatter(
            x=dff['Dprio'].tolist(),
            y=dff[i].tolist(),
            mode = 'lines',
            name = name_dict[i],
            line = dict(
              dash = linedash_dict[i],
              color = linecolor_dict[i],
              width = 2
           )
    ))

    
    return {
    'data': traces,
    'layout': go.Layout(
        xaxis={'title': 'Days before departure', 'range': [dff['Dprio'].min(), 365]},
        yaxis={'title': 'Number of bookings', 'range': [0, max([dff['Phase-down frontier'].max(),dff['Actual Bookings'].max()])]})
    }
    
    #return generate_table(dff)

In [ ]:
@app.callback(
    Output('table-container', 'children'),
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_table(value_depdates, value_dtime):

    dff = df.loc[(df['DepDate'] == value_depdates)&(df['dtime'] == value_dtime),
        ['Dprio', 'Actual Bookings', 'Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 'Phase-down frontier']] # update with your own logic    
    dff.columns=[x.replace(' ','_') for x in dff.columns]
    
    return dash_table.DataTable(
        data=dff.to_dict('rows'),
        columns=[{'id': c, 'name': c.replace('_',' ')} for c in dff.columns],
        # style_as_list_view=True,
        style_header={'backgroundColor': 'rgb(30, 30, 30)','color': 'white', 'textAlign': 'center'},
#         style_cell={
#             'backgroundColor': 'rgb(50, 50, 50)',
#             'color': 'white'
#         },
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(40, 40, 40)',
            'color': 'white',
            'textAlign': 'center'
        },
        {
            'if': {'row_index': 'odd', 'column_id': 'Dprio'},
            'backgroundColor': '#FFFF99',
            'color': 'black',
            'textAlign': 'center'
        },    
        {
            'if': {'row_index': 'even'},
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white',
            'textAlign': 'center'
        },
        {
            'if': {'row_index': 'even', 'column_id': 'Dprio'},
            'backgroundColor': '#FFFFCC',
            'color': 'black',
            'textAlign': 'center'
        },     
        {
             'if': {
                 'column_id': 'Actual_Bookings',
                 'filter': 'Actual_Bookings < Ramp-up_frontier',
             },
            'color': 'red',
            'textAlign': 'left'
        },
        {
            'if': {
                 'column_id': 'Ramp-up_frontier',
                 'filter': 'Actual_Bookings < Ramp-up_frontier',
             },
            'color': 'red'
        },
        {
            'if': {
                 'column_id': 'Actual_Bookings',
                 'filter': 'Actual_Bookings > Phase-down_frontier',
             },
            'color': 'red',
            'textAlign': 'right'
        },
        {
            'if': {
                 'column_id': 'Phase-down_frontier',
                 'filter': 'Actual_Bookings > Phase-down_frontier',
             },
            'color': 'red'
        }],
        style_cell={
        # all three widths are needed
        'minWidth': '75px', 'width': '75px', 'maxWidth': '75px',
        'whiteSpace': 'normal'
    }
        )

In [ ]:
if __name__ == '__main__':
    app.run_server(port=1000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2019 23:47:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:47:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 23:51:39] "POST /_dash-update-component